<a href="https://colab.research.google.com/github/team05-MLOps-pipeline/kafka-consumer-db/blob/main/kafka_consumer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install confluent-kafka mysql-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.8/409.8 kB 33.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.21.12 which is incompatible.


In [2]:
!pip install pymysql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.3 MB/s eta 0:00:00


In [6]:
from confluent_kafka import Consumer, KafkaException
import pymysql
import datetime
import json
from datetime import datetime

# Kafka consumer 설정
conf = {'bootstrap.servers': 'shtestdb.duckdns.org:9094',
        'group.id': 'mygroup',
        'auto.offset.reset': 'earliest'}
consumer = Consumer(conf)

# MySQL 연결 설정
conn = pymysql.connect(host='shtestdb.duckdns.org',
                       user='eun',
                       port=13306,
                       password='12341234!',
                       db='eun_test',
                       charset='utf8')
cursor = conn.cursor()

def convert_datetime_format(get_time):
    """
    2023.08.31. 오후 3:51
    to
    YYYY-MM-DD HH:MI:SS
    """
    if '오후' in get_time:
        # 오후인 경우, 시간에 12를 더해서 24시간 형식으로 변환
        tmp_time = datetime.strptime(get_time, "%Y.%m.%d. 오후 %I:%M")
        tmp_time = tmp_time.replace(hour=tmp_time.hour + 12)
    else:
        # 오전인 경우, 그대로 24시간 형식으로 변환
        tmp_time = datetime.strptime(get_time, "%Y.%m.%d. 오전 %I:%M")

    return tmp_time.strftime("%Y-%m-%d %H:%M:%S")

no_message_count = 0

try:
    # Kafka topic 구독
    consumer.subscribe(['euntest'])

    while True:
        msg = consumer.poll(3.0)  # timeout is set to 1 second

        if msg is None:
            no_message_count += 1

            if no_message_count > 5:
                print("No message received... Exiting")
                break

            continue

        else:
            no_message_count=0

        if msg.error():
            raise KafkaException(msg.error())

        else:
            value = msg.value().decode('utf-8')
            print('Received message: {}'.format(value))

            # MySQL에 데이터 적재하기
            sql = "INSERT INTO Article (category_id, news_url, company_name, title, author_info, create_date, modify_date, content, image_url) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)"

            data = json.loads(value)  # JSON 문자열을 Python 객체로 변환

            print(data)

            create_date_converted=convert_datetime_format(data['create_date'])

            try:
                cursor.execute(sql,
                            (data['category'], data['url'], data['site_name'], data['title'],
                                data['author_info'], create_date_converted , None,
                                data['content'], data['image_url']))
                conn.commit()  # 변경 사항 커밋
            except pymysql.err.IntegrityError as e:
                if e.args[0] == 1062:  # duplicate entry error code
                    print(f"Duplicate entry {data['url']} skipped.")
                else:
                    raise e  # re-raise the exception if it's not due to a duplicate entry



except KeyboardInterrupt:
    pass

finally:
    # Close down consumer to commit final offsets.
    consumer.close()

    # MySQL 연결 종료
    cursor.close()
    conn.close()






Received message: {"category": 100, "url": "https://n.news.naver.com/mnews/article/021/0002598993?sid=100", "site_name": "\ubb38\ud654\uc77c\ubcf4", "title": "\u8207 \uc1a1\uc11d\uc900 \u201c\uc911\uacf5\uad70 \uce68\ub7b5 \uc815\ub2f9\ud654 \uc815\uc728\uc131, \uc758\uc778\uc778 \uc591 \uae30\ub9ac\ub294 \uac8c \ub9d0\uc774 \ub418\ub098\u201d", "author_info": "\uc815\ucda9\uc2e0 \uc120\uc784\uae30\uc790(csjung@munhwa.com)", "create_date": "2023.10.13. \uc624\ud6c4 4:43", "modify_date": null, "content": "\uc815\uc728\uc131\uc774 \uc7a0\uc2dc \ub2e4\ub154\ub2e4\ub294 \uc804\ub0a8 \ud654\uc21c\uad70 \ub2a5\uc8fc\ucd08\ub4f1\ud559\uad50 \uc678\ubcbd\uc5d0 \uadf8\ub824\uc9c4 \uc815\uc728\uc131\uc758 \ub300\ud615 \ubcbd\ud654. \ud654\uc21c\uad70\uc774 \ub2a5\uc8fc\ucd08 \uad50\uc7a5 \uc694\uccad\uc73c\ub85c \ucca0\uac70\ub97c \uac80\ud1a0 \uc911\uc774\ub2e4. \ud654\uc21c\uad70 \ud648\ud398\uc774\uc9c0 \ucea1\ucc98\u25a0 \uc815\uc728\uc131 \uacf5\uc6d0\u00b7\ud64d\ubc94\ub3c4 \ud749\uc0c1\u2

In [10]:
# 24시간이 넘는 수를 처리해주는 코드
# 종료코드 적용 전

from confluent_kafka import Consumer, KafkaException
import pymysql
import datetime
import json

# Kafka consumer 설정
conf = {'bootstrap.servers': 'shtestdb.duckdns.org:9094',
        'group.id': 'mygroup',
        'auto.offset.reset': 'earliest'}
consumer = Consumer(conf)

# MySQL 연결 설정
conn = pymysql.connect(host='shtestdb.duckdns.org',
                       user='eun',
                       port=13306,
                       password='12341234!',
                       db='eun_test',
                       charset='utf8')
cursor = conn.cursor()

def convert_date(kr_date_str):
    # "2023.10.13. 오후 1:09" --> "2023-10-13 13:09"
    date_str, am_pm, time_str = kr_date_str.split()
    hour, minute = map(int, time_str.split(':'))

    if am_pm == "오후" and hour < 12:
        hour += 12
    elif am_pm == '오전' and hour == 12:
        hour = 0

    return f"{date_str.replace('.', '-')} {hour:02d}:{minute:02d}:00"

try:
    # Kafka topic 구독
    consumer.subscribe(['euntest'])

    while True:
        msg = consumer.poll(1.0)  # timeout is set to 1 second

        if msg is None:
            continue
        if msg.error():
            raise KafkaException(msg.error())
        else:
            value = msg.value().decode('utf-8')
            print('Received message: {}'.format(value))

            # MySQL에 데이터 적재하기
            sql = "INSERT INTO Article (category_id, news_url, company_name, title, author_info, create_date,\
                   modify_date,content,image_url) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)"

            data=json.loads(value)

            create_date_converted=convert_date(data['create_date'])

            # try:
            #     cursor.execute(sql,
            #                    (data['category'], data['url'], data['site_name'], data['title'],
            #                     data['author_info'], create_date_converted , None,
            #                     data['content'], data['image_url']))
            #     conn.commit()

            # except pymysql.err.IntegrityError as e:
            #     if e.args[0] == 1062:
            #         print(f"Duplicate entry {data['url']} skipped.")
            #     else:
            #         raise e
            cursor.execute(sql,
                           (data['category'], data['url'], data['site_name'], data['title'],
                            data['author_info'], create_date_converted , None,
                            data['content'], data['image_url']))

            conn.commit()  # 변경 사항 커밋

except KeyboardInterrupt:
    pass

finally:

   consumer.close()
   cursor.close()
   conn.close()